# English detector 
## English or not english ?

In [2]:
import nltk
import nltk.corpus
import nltk.data
import random
from pprint import pprint
from collections import Counter

## Downloading the corpus

> For this project I have decide to use the Universal Declaration of the Human Right because it's translated in many languages, for this assesment it's very useful.

In [3]:
nltk.download('udhr2')

[nltk_data] Downloading package udhr2 to /home/stromax/nltk_data...
[nltk_data]   Package udhr2 is already up-to-date!


True

**To see all the texts of the corpus we can use this :**

In [4]:
#udhr.fileids()

## Udhr Languages :
> For the project I decide to use four languages:  

- English  
- French  
- Italian  
- Esperanto  

>In this section I get the words and the sentences of thoses texts.  
I will explain later why I need to get the words and the sentences of the texts.

In [5]:
#Words
english_udhr_words = nltk.corpus.udhr2.words('eng.txt')
french_udhr_words = nltk.corpus.udhr2.words('fra.txt')
italian_udhr_words = nltk.corpus.udhr2.words('ita.txt')
esperanto_udhr_words = nltk.corpus.udhr2.words('epo.txt')



#Sentences
english_udhr_sents = nltk.corpus.udhr2.sents('eng.txt')
french_udhr_sents = nltk.corpus.udhr2.sents('fra.txt')
italian_udhr_sents = nltk.corpus.udhr2.sents('ita.txt')
esperanto_udhr_sents = nltk.corpus.udhr2.sents('epo.txt')

#Printing the numbers of words in the different languages
print("English words :", len(english_udhr_words))
print("French words :", len(french_udhr_words))
print("Italian words :", len(italian_udhr_words))
print("Esperanto words :", len(esperanto_udhr_words),"\n")

#Printing the numbers of sentences in the different languages
print("English sentences :", len(english_udhr_sents))
print("French sentences :", len(french_udhr_sents))
print("Italian sentences :", len(italian_udhr_sents))
print("Esperanto sentences :", len(esperanto_udhr_sents))



English words : 1918
French words : 2316
Italian words : 2173
Esperanto words : 1833 

English sentences : 74
French sentences : 74
Italian sentences : 77
Esperanto sentences : 75


## Words and sentences

> In this section I do two things : 
First, I add all words of the different languages in a list called *languages_words*.    
Second, I do the same things for the sentences but I need to add label to each sentences in function of their language here is the labels used : 

- english   
- french   
- italian   
- esperanto     



In [8]:
languages_words = []

#Adding english sentences
for en in english_udhr_words:
   languages_words.append(en)
    
# Adding french sentences
for fr in french_udhr_words:
   languages_words.append(fr)
    
# Adding italian sentences
for it in italian_udhr_words:
   languages_words.append(it)

# Adding italian sentences
for ep in esperanto_udhr_words:
   languages_words.append(ep)

languages_sents = []

#Adding english sentences
for en in english_udhr_sents:
   languages_sents.append((en, 'english'))
    
# Adding french sentences
for fr in french_udhr_sents:
   languages_sents.append((fr, 'french'))
    
# Adding italian sentences
for it in italian_udhr_sents:
   languages_sents.append((it, 'italian'))

# Adding italian sentences
for ep in esperanto_udhr_sents:
   languages_sents.append((ep, 'esperanto'))

    
#Shuffle the list of sentences
random.shuffle(languages_sents)

### Creating the features

> Now I will use this function to format a sentences in a way that they can be used by the classifier.  
The reason why I needed a list of words is because this list will be used as keys for this function.

>The function will compare the list of words and the list of sentences to tell what words is contained in a sentence, if a word is contained it will render **True** otherwise **False**.


In [9]:
word_features = nltk.FreqDist(languages_words).keys()

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features



## Splitting the list to obtain the train and the test set

> Here I will split the features set previously obtained to get a train set and a test set the distribution will be as follows :  
>- **80%** of the feature set for the train set  
>- **20%** of the feature set for the test set  

In [10]:
size_80 = int(len(languages_sents) * 0.8)

train_set = languages_sents[:size_80]
test_set = languages_sents[size_80:]

print("Number of sentences in the training set :",len(train_set))
print("Number of sentences in the test set :",len(test_set))

train_featured_set = nltk.classify.apply_features(extract_features, train_set)
test_featured_set = nltk.classify.apply_features(extract_features, test_set)


Number of sentences in the training set : 240
Number of sentences in the test set : 60


### Preparing the training of the classifier

> And finally I will train the classifier !

In [11]:

classifier = nltk.NaiveBayesClassifier.train(train_featured_set)

# Testing the classifier

> Now that the classifier is trained I have done some examples to the reliability of it, I have also shown the accuracy calculated by nltk for the train set and the test set

In [12]:

print("Accuracy train set :",nltk.classify.accuracy(classifier, train_featured_set))
print("Accuracy test set :",nltk.classify.accuracy(classifier, test_featured_set))

t1 = "The sudden departure of the head of trust and safety, the chief information security officer, and both the chief privacy and compliance officers is a dramatic development. It's not clear how soon they will be replaced, while the firm remains this unstable and sweeping job cuts have already been made."
t2 = "La presqu’île de Gâvres, près de Lorient (Morbihan), est l’un des sites de Bretagne les plus vulnérables face à la montée des eaux. Selon les scientifiques, la route qui la relie au continent sera sous l’eau en 2100 et une centaine de maisons disparaîtront. Mais la commune continue d’attirer et projette de construire 37 maisons."
t3 = "Italia, che spesso lo acquisiscono e lo usano insieme alle varianti regionali dell'italiano, alle lingue regionali e ai dialetti. In Italia viene ampiamente usato per tutti i tipi di comunicazione della vita quotidiana ed è largamente prevalente nei mezzi di comunicazione nazionali, nell'amministrazione pubblica dello Stato italiano e nell'editoria."
t4 = "Esperanto estas la plej sukcesa konstruita internacia helplingvo, kaj la sola tia lingvo kun ampleksa loĝantaro de denaskaj parolantoj, el kiuj estas eble kelkaj miloj. Uzadotaksoj estas malfacilaj, sed du taksoj metas la nombron de homoj kiuj scipovas paroli Esperanton je proksimume 100 000. Koncentriĝo de parolantoj estas plej alta en Eŭropo, Orienta Azio, kaj Sudameriko."


print(classifier.classify(extract_features(t1.split())))
print(classifier.classify(extract_features(t2.split())))
print(classifier.classify(extract_features(t3.split())))
print(classifier.classify(extract_features(t4.split())))


Accuracy train set : 0.9291666666666667
Accuracy test set : 0.9
english
french
italian
esperanto


## The confusion matrix

> For the end I will do a confusion matrix but for this I need two arrays : 
- gold :  which will be the true values of the confusion matrix
- system : which will be the values of the system

>I fill the gold array with the train set( named *train_featured_set*) because this is the values that have been trained with the classifier and I fill the system array with the test set(named *test_featured_set*).

>After that I just need to use the confusion matrix method from nltk.



In [13]:
gold = []

for i in range(len(train_featured_set)):
    for j in range(len(train_featured_set[i])):
        gold.append(train_featured_set[i][1])

system = []

for i in range(len(test_featured_set)):
    for j in range(len(test_featured_set[i])):
        system.append(test_featured_set[i][1])

print(len(gold))
print(len(system))

cm = nltk.ConfusionMatrix(gold[:len(test_featured_set)*2], system)

print(cm)


480
120
          |     e       |
          |     s       |
          |  e  p     i |
          |  n  e  f  t |
          |  g  r  r  a |
          |  l  a  e  l |
          |  i  n  n  i |
          |  s  t  c  a |
          |  h  o  h  n |
----------+-------------+
  english |<14> 4  6 12 |
esperanto |  6 <4> 6 10 |
   french |  6 10 <8> 6 |
  italian |  8  6  6 <8>|
----------+-------------+
(row = reference; col = test)



## The precision, the accuracy and the recall

> Finally we will measures the precision, the accuracy and the recall but for this we need to get the values of true positives, true negatives, false positives and false negatives.

>For that I create a set which contain the labels and I go through it with 4 loops : 
>The first loop (i) get a row value ( for example **English**).

>The second loop (j) will go through that row and when i and j is equal *( For example i is **English** and j is **English** )* I add the value of the confusion matrix to the true positives.   
If i and j are not equal *( For example i is **English** and j is **French** )* then I add the value of the confusion matrix to the "i" values of the false negatives ( because false negatives are on rows) and I add the values of the confusion matrix to the "j" values of the false positives (because false positives are on columns).

> The third loop and fourth loop is used when we have a true positive, at this moment we will go through the confusion matrix another time, k will works in the same way of i and l will works in the same way of j. To add  the true negative I proceed in this way, a true negative value is a value that is not a true positive, a false negative and a false postive, for the confusion matrix I have deduced that this a value which dont have **English** for i and j, for example. So a true negative will be add when the values of k and l will be different for i and j ( For example i and j is **English** so when k and l is different from **English** then we add a true negative values for **English**).


> After that I have all of the values needed to calculated the precision, the accuracy and the recall with thoses formulas:  
- Precision : **True positive / True positive + False positive**  
- Recall : **True positive / True positive + False negative**  
- Accuracy : **True positive + True negative/ True positive + True negative + False positive + False negative**  

In [14]:
labels = set('french english esperanto italian'.split())

true_pos = Counter()
true_neg = Counter()
false_pos = Counter() 
false_neg = Counter() 

for i in labels:
    for j in labels:
        if i == j:
            true_pos[i] += cm[i,j]
            for k in labels:
                if k != i:
                    for l in labels:
                        if k!=i and l!= j:
                            true_neg[i]+= cm[k,l]
        else:
            false_neg[i] += cm[i,j]
            false_pos[j] += cm[i,j]
            

true_pos_total = sum(true_pos.values())
true_neg_total = sum(true_neg.values())
false_pos_total = sum(false_pos.values())
false_neg_total = sum(false_neg.values())


print("Precision :",(true_pos_total/(true_pos_total+false_pos_total)))
print("Recall :",(true_pos_total/(true_pos_total+false_neg_total)))
print("Accuracy :",((true_pos_total+ true_neg_total)/(true_pos_total+true_neg_total+false_neg_total+false_neg_total)))



Precision : 0.2833333333333333
Recall : 0.2833333333333333
Accuracy : 0.6416666666666667


***That's All Folks !***  
Alan Vadeleau - French erasmus student
